In [154]:
from dotenv import load_dotenv
import os

load_dotenv(verbose=True)
key = os.getenv('OPENAI_API_KEY')

In [155]:
from langchain_teddynote.tools.tavily import TavilySearch

tavily_tool = TavilySearch(max_results=1)           # 검색 도구 정의(TavilySearch)

In [156]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.tools import tool
from typing import List

@tool
def scrape_webpages(urls: List[str]) -> str:
    """Use requests and bs4 to scrape the provided web pages for detailed information."""
    
    # 주어진 URL 목록을 사용하여 웹 페이지 로드
    loader = WebBaseLoader(
        web_path=urls,
        header_template={
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36",
        },
    )
    
    docs = loader.load()                        # 로드된 문서

    result = []

    for doc in docs:
        title = doc.metadata.get("title", "")   # 제목
        content = doc.page_content              # 본문
        
        document_string = f'<Document name="{title}">\n{content}\n</Document>'

        result.append(document_string)          # 제목과 내용을 포함한 문자열 

    final_string = "\n\n".join(result)          # 문자열로 만든다

    return final_string    

In [157]:
# tool 사용 예시
urls_to_scrape = [
    "https://n.news.naver.com/article/015/0005106146",
]

scraped_content = scrape_webpages.invoke({'urls': urls_to_scrape})
print(scraped_content)

<Document name=""06년생 라오스女와 결혼"…韓 3040 남성들 눈 돌린 곳이 [요즘 결혼(끝)]">




















"06년생 라오스女와 결혼"…韓 3040 남성들 눈 돌린 곳이 [요즘 결혼(끝)]














본문 바로가기






이전 페이지










한국경제





구독

메인 뉴스판에서 한국경제 주요뉴스를  볼 수 있습니다.
보러가기


한국경제 언론사 구독 해지되었습니다.










주요뉴스
프리미엄
이슈
숏폼
경제
정치
사회
IT
생활
세계
사설/칼럼
신문보기
생중계
랭킹















한국경제

한국경제



PICK
안내


언론사가 주요기사로선정한 기사입니다.
언론사별 바로가기
닫기




"06년생 라오스女와 결혼"…韓 3040 남성들 눈 돌린 곳이 [요즘 결혼(끝)]




입력2025.03.14. 오후 2:59


수정2025.03.14. 오후 3:17

기사원문
 

유지희 기자
신현보 기자








유지희 기자




유지희 기자

구독
구독중




구독자
0


응원수
0



더보기







신현보 기자




신현보 기자

구독
구독중




구독자
0


응원수
0



더보기














추천




쏠쏠정보
0




흥미진진
0




공감백배
0




분석탁월
0




후속강추
0


 



댓글





본문 요약봇



본문 요약봇도움말
자동 추출 기술로 요약된 내용입니다. 요약 기술의 특성상 본문의 주요 내용이 제외될 수 있어, 전체 맥락을 이해하기 위해서는 기사 본문 전체보기를 권장합니다.
닫기








텍스트 음성 변환 서비스 사용하기



성별
남성
여성


말하기 속도
느림
보통
빠름

이동 통신망을 이용하여 음성을 재생하면 별도의 데이터 통화료가 부과될 수 있습니다.
본문듣기 시작

닫기


 

글자 크기 변경하기



가1단계
작게


가2단계
보통


가3단계
크게


가4단계
아주크게




In [158]:
from pathlib import Path
from typing import Dict, Optional, List
from typing_extensions import Annotated


WORKING_DIRECTORY = Path("./tmp")           # 임시 디렉토리 생성 및 작업 디렉토리 설정
WORKING_DIRECTORY.mkdir(exist_ok=True)      # tmp 폴더가 없으면 생성

### 아웃라인 생성 및 파일로 저장

In [159]:
@tool
def create_outline(
    points: Annotated[List[str], "List of main points or sections."],
    file_name: Annotated[str, "File path to save the outline."],
) -> Annotated[str, "Path of the saved outline file."]:
    
    """Create and save an outline."""

    # 주어진 파일 이름으로 아웃라인을 저장
    with (WORKING_DIRECTORY / file_name).open("w", encoding='utf-8') as file:
        
        for i, point in enumerate(points):
            file.write(f"{i + 1}. {point}\n")
            
    return f"Outline saved to {file_name}"

In [160]:
outline_points = [
        "각각의 Outline 에 대해서 5문장 이상 작성",
        "한글로 리포트 작성",
        "출처를 작성",
    ]

outline_file_name = "my_outline.txt"

create_outline.invoke({'points': outline_points, 'file_name': outline_file_name})

'Outline saved to my_outline.txt'

### 문서 읽기

In [161]:
@tool
def read_document(
    file_name: Annotated[str, "File path to read the document."],
    start: Annotated[Optional[int], "The start line. Default is 0"] = None,
    end: Annotated[Optional[int], "The end line. Default is None"] = None,
) -> str:
    
    """Read the specified document."""

    # 주어진 파일 이름으로 문서 읽기
    with (WORKING_DIRECTORY / file_name).open("r", encoding='utf-8') as file:
        lines = file.readlines()

    
    if start is not None:               # 시작 줄이 지정되지 않은 경우 기본값 설정
        start = 0

    return "\n".join(lines[start:end])

In [162]:
read_document.invoke({'file_name': outline_file_name, 'start':0, 'end':3})

'1. 각각의 Outline 에 대해서 5문장 이상 작성\n\n2. 한글로 리포트 작성\n\n3. 출처를 작성\n'

### 문서 쓰기 및 저장

In [163]:
@tool
def write_document(
    content: Annotated[str, "Text content to be written into the document."],
    file_name: Annotated[str, "File path to save the document."],
) -> Annotated[str, "Path of the saved document file."]:
    
    """Create and save a text document."""

    # 주어진 파일 이름으로 문서 저장
    with (WORKING_DIRECTORY / file_name).open("w", encoding='utf-8') as file:
        file.write(content)

    return f"Document saved to {file_name}"

In [164]:
write_document.invoke({'content': '문서 저장하기', 'file_name': 'a.txt'})

'Document saved to a.txt'

### 문서 편집

In [165]:
@tool
def edit_document(
    file_name: Annotated[str, "File path of the document to be edited."],
    inserts: Annotated[
        Dict[int, str], 
        "Dictionary where key is the line number (1-indexed) and value is the text to be inserted at that line."
        ],
) -> Annotated[str, "File path of the edited document."]:
    
    """Edit a document by inserting text at specific line numbers."""

    # 주어진 파일 이름으로 문서 읽기
    with (WORKING_DIRECTORY / file_name).open("r", encoding='utf-8') as file:
        lines = file.readlines()

    # 삽입할 텍스트를 정렬하여 처리
    sorted_inserts = sorted(inserts.items())

    print(sorted_inserts)

    # 지정된 줄 번호에 텍스트 삽입
    for line_number, text in sorted_inserts:
        print(line_number, text)

        if 1 <= line_number <= len(lines) + 1:
            lines.insert(line_number - 1, text + "\n")
        else:
            return f"Error: Line number {line_number} is out of range."

    print(lines)
    
    # # 편집된 문서를 파일에 저장
    with (WORKING_DIRECTORY / file_name).open("w", encoding='utf-8') as file:
        file.writelines(lines)

    return f"Document edited and saved to {file_name}"

In [166]:
test_inserts = {
        1: "[추가 문장1]",  # 첫 번째 줄에 삽입
        3: "[추가 문장3]"   # 기존 3번째 줄 다음에 삽입
    }

edit_document.invoke({'file_name': 'test_document.txt', 'inserts': test_inserts})

FileNotFoundError: [Errno 2] No such file or directory: 'tmp\\test_document.txt'

In [ ]:
from langchain_experimental.tools import PythonREPLTool

python_repl_tool = PythonREPLTool()     # PythonREPL 도구

### State 정의

In [ ]:
import operator
from typing import List, TypedDict
from typing_extensions import Annotated
from langchain_core.messages import BaseMessage


class ResearchState(TypedDict):
    messages: Annotated[List[BaseMessage], operator.add]    # 메시지
    team_members: List[str]                                 # 멤버 에이전트 목록
    next: str                                               # Supervisor 에이전트에게 다음 작업자를 선택하도록 지시

### 에이전트 팩토리 클래스

In [ ]:
from langgraph.graph import START, END
from langchain_core.messages import HumanMessage
from langchain_openai.chat_models import ChatOpenAI
from langgraph.prebuilt import create_react_agent


class AgentFactory:
    def __init__(self, model_name='gpt-4o-mini'):
        self.llm = ChatOpenAI(api_key=key, model=model_name, temperature=0)

    def create_agent_node(self, agent, name: str):              # 노드 생성 함수
        print(f'create_agent_node() 호출: {name} 노드 생성')

        def agent_node(state):      
            print(f'agent_node() 함수 실행')     

            result = agent.invoke(state)

            print(f'사용된 이름: {name}')
            print(result["messages"][-1].content)

            return {
                "messages": [
                    HumanMessage(content=result["messages"][-1].content, name=name)
                ]
            }

        return agent_node


llm = ChatOpenAI(
    api_key=key, 
    model='gpt-4o', 
    temperature=0
)


# Agent Factory 인스턴스 생성
agent_factory = AgentFactory('gpt-4o-mini')

In [ ]:
# 에이전트 정의
search_agent = create_react_agent(llm, tools=[tavily_tool])

# 에이전트 노드 생성
search_node = agent_factory.create_agent_node(search_agent, name="Searcher")

create_agent_node() 호출: Searcher 노드 생성


In [ ]:
from langchain_core.runnables import RunnableLambda
from langchain_core.messages import HumanMessage

# 1. Runnable로 래핑하여 invoke 지원 활성화
search_runnable = RunnableLambda(search_node)

# 2. 실행할 상태 객체 생성 (ResearchState 구조 준수)
research_state = {
    "messages": [HumanMessage(content="대구 동성로 떡볶이 맛집 추천")],
    "team_members": ["Searcher"],
    "next": "Searcher"
}

# 3. 에이전트 노드 실행
result = search_runnable.invoke(research_state)
print(result)

agent_node() 함수 실행
사용된 이름: Searcher
대구 동성로에서 떡볶이 맛집으로 추천할 만한 곳은 '중앙떡볶이'입니다. 이곳은 매콤한 떡볶이와 납작 만두가 맛있기로 유명합니다. 더 많은 정보를 원하시면 [여기](https://hotplacehunter.co.kr/local/article/56321/)를 참고하세요.
{'messages': [HumanMessage(content="대구 동성로에서 떡볶이 맛집으로 추천할 만한 곳은 '중앙떡볶이'입니다. 이곳은 매콤한 떡볶이와 납작 만두가 맛있기로 유명합니다. 더 많은 정보를 원하시면 [여기](https://hotplacehunter.co.kr/local/article/56321/)를 참고하세요.", additional_kwargs={}, response_metadata={}, name='Searcher')]}


In [ ]:
# 웹 스크래핑 노드 생성
web_scraping_agent = create_react_agent(llm, tools=[scrape_webpages])

web_scraping_node = agent_factory.create_agent_node(
    web_scraping_agent, name="WebScraper"
)

create_agent_node() 호출: WebScraper 노드 생성


In [ ]:
from langchain_core.runnables import RunnableLambda
from langchain_core.messages import HumanMessage

# 1. Runnable로 래핑하여 invoke 지원 활성화
scraping_runnable = RunnableLambda(web_scraping_node)

# 2. 실행할 상태 객체 생성 (ResearchState 구조 준수)
scraping_state = {
    "messages": [HumanMessage(content="https://n.news.naver.com/article/015/0005106146")],
    "team_members": ["WebScraper"],
    "next": "Searcher"
}

# 3. 에이전트 노드 실행
result = scraping_runnable.invoke(scraping_state)
print(result)

agent_node() 함수 실행
사용된 이름: WebScraper
The article from Naver News discusses the increasing trend of international marriages among South Korean men, particularly those in their 30s and 40s. The rising costs of traditional weddings in South Korea, often referred to as "wedding inflation," have led many to consider more cost-effective alternatives, such as international marriages. The article highlights that the average cost of a traditional wedding in South Korea is significantly higher than that of an international marriage, which can be as low as 1/14th of the domestic cost.

The article also notes a shift in the demographics of those seeking international marriages, with more young professionals in urban areas considering this option. The number of international marriage agencies has increased, and the interest in international marriages is growing, as evidenced by the popularity of related content on platforms like YouTube.

Additionally, the article provides statistics on the increa

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI
from pydantic import BaseModel
from typing import Literal


def create_team_supervisor(model_name, system_prompt, members) -> str:
    
    options_for_next = ["FINISH"] + members

    # 작업자 선택 응답 모델 정의: 다음 작업자를 선택하거나 작업 완료를 나타냄
    class RouteResponse(BaseModel):
        next: Literal[*options_for_next]


    # ChatPromptTemplate 생성
    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", system_prompt),
            MessagesPlaceholder(variable_name="messages"),
            (
                "system",
                "Given the conversation above, who should act next? "
                "Or should we FINISH? Select one of: {options}",
            ),
        ]
    ).partial(options=str(options_for_next))
    
    
    llm = ChatOpenAI(
        api_key=key, 
        model=model_name, 
        temperature=0
    )

    # 프롬프트와 LLM을 결합하여 체인 구성
    supervisor_chain = prompt | llm.with_structured_output(RouteResponse)

    return supervisor_chain

In [ ]:
# Supervisor 에이전트 생성
supervisor_agent = create_team_supervisor(
    'gpt-4o',
    "You are a supervisor tasked with managing a conversation between the"
    " following workers: Search, WebScraper. Given the following user request,"
    " respond with the worker to act next. Each worker will perform a"
    " task and respond with their results and status. When finished,"
    " respond with FINISH.",
    ["Searcher", "WebScraper"],
)

In [ ]:
from langchain_core.messages import HumanMessage, AIMessage

# 초기 메시지 구성 (에이전트 협업 시나리오)
messages = [
    HumanMessage(content="대구 동성로 떡볶이 맛집 2개를 찾아 분석해주세요"),
    AIMessage(
        content="Searcher가 웹 검색을 수행했습니다: 신전떡볶이, 엽기떡볶이 발견",
        name="Searcher"
    )
]

# Supervisor 호출
decision = supervisor_agent.invoke({
    "messages": messages  # 필수 키: messages
})

# 결과 해석
print(f"{decision.next}")
print(f"{decision}")


WebScraper
next='WebScraper'


In [ ]:
def get_next_node(x):
    return x["next"]

[ERROR] Visualize Graph Error: HTTPSConnectionPool(host='mermaid.ink', port=443): Read timed out. (read timeout=10)
